In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
treino = pd.read_csv("../input/mnist-digit-recognizer/train.csv")

In [ ]:
y = treino['label']
x = treino.drop(labels=['label'],axis=1)
x = x/255.0

In [ ]:
x = x.values.reshape(-1,28,28,1)
y = to_categorical(y, num_classes=10)

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=.2)

In [ ]:
modelo = Sequential()
modelo.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (28,28,1)))
modelo.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu'))
modelo.add(MaxPool2D(pool_size=(2,2)))
modelo.add(Dropout(0.25))
modelo.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
modelo.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
modelo.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
modelo.add(Dropout(0.25))
modelo.add(Flatten())
modelo.add(Dense(128, activation = "relu"))
modelo.add(Dropout(0.5))
modelo.add(Dense(10, activation = "softmax"))

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
modelo.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
redutor = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
parada = EarlyStopping(monitor='val_accuracy', patience=10, mode='min')

In [ ]:
aug = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,
        rotation_range=3,  # Gira a imagem em 3 graus
        zoom_range = 0.1, # Da um zoom na imagem de forma aleatória
        width_shift_range=0.1,  # Move a imagem de forma aleatória horizontalmente
        height_shift_range=0.1)  # Move a imagem de forma aleatória verticalmente
aug.fit(x_treino)

In [ ]:
epochs = 100
batch_size = 64

In [ ]:
modelo = modelo.fit(aug.flow(x_treino, y_treino, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_teste, y_teste),
                              verbose = 1, steps_per_epoch=x_treino.shape[0] // batch_size
                              , callbacks=[redutor,parada])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].plot(modelo.history['accuracy'])
axes[0].plot(modelo.history['val_accuracy'])
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Acurácia')
axes[0].legend(['Acurácia em Treino','Acurácia em Teste'])
axes[0].grid(True)

axes[1].plot(modelo.history['loss'])
axes[1].plot(modelo.history['val_loss'])
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Erro')
axes[1].legend(['Erro em Treino','Erro em Teste'])
axes[1].grid(True)